In [ ]:
from pandas import *
from numpy import *
from numpy.linalg import * 
from matplotlib.pylab import *

# Bac à sable pour les données "jouet"

In [ ]:
X=array([[12,16,14],[14,13,15],[8,11,8],[10,9,6],[16,6,17]]) # rappel des données jouet

# Classification hiérarchique de Ward

On va travailler avec le fichier `depenses-etat.xls` qui contient la répartition des dépenses de l'État  français par ministère entre 1872 et 1971. Ces données sont tirées de l'ouvrage *L'Etat et l'Economie. Essai d'explication de l'évolution des dépenses publiques en France (1870-1980)* de  Christine Andre et  Robert Delorme. 

In [ ]:
df = read_excel('depenses-etat.xls')
df.head(10)

On crée les array `annees`, `ministeres` et `data` contenant respectivement les années, les noms des ministères
et les valeurs du tableau de données.

In [ ]:
annees = df['années'].values
data = df.values
data = delete(data,[0],axis=1)
ministeres = asarray(df.columns)
ministeres=delete(ministeres,[0])
print(annees)
print(ministeres)
print(shape(data),len(annees),len(ministeres))

On a recopié ci-dessous la fonction `normalise` et la fonction `acp` écrites lors du TP précédent.

In [ ]:
def normalise(M):
    n,p=shape(M)
    Z=zeros((n,p))
    for j in range(p):
        Z[:,j]=(M[:,j]-mean(M[:,j]))/std(M[:,j])
    return Z
def acp(M):
    n,p=shape(M) 
    Z=normalise(M)
    R= 1/n*dot(Z.T,Z)
    val0, P0 = eigh(R) 
    val = sort(val0)[::-1] 
    index = argsort(val0)[::-1] 
    P=zeros((p,p))
    for j in range(p):
        P[:,j]=P0[:,index[j]]
    C=dot(Z,P)
    return val, P, C

**Question 1.** Représentez la projection des individus (ici les années) dans le plan principal. Affectez à chaque point du plan l'année correspondante, en utilisant la commande `annotate(annees[i],(C1[i],C2[i]))`, où `C1` et `C2` désignent les deux premières composantes principales. Ajoutez l'instruction
`axhline(y=0, lw=1, color='k'); axvline(x=0, lw=1, color='k')` afin d'afficher sur le graphique les deux axes Ox et Oy correspondant à C1 et C2. 

Commentez ce graphique.

Nous avons vu qu'il semblait naturel, après avoir effectué une ACP, de regrouper les dépenses de l'état en 4 clusters. Nous allons donc programmer l'algorithme de classification (non supervisé) de Ward, pour effectuer une classification automatique. Cela nous permettra entre autres de juger de la pertinence de ces 4 clusters.

**Question 3.** Créez une fonction `bary` qui prend en arguments
- deux arrays `G1` et `G2` qui sont les centres de gravité de deux classes,
- deux nombres `n1` et `n2` qui indiquent le nombre d'individus dans chaque classe,

et renvoie les coordonnées du centre de gravité entre les classes de centres de gravité `G1` et `G2`.

Pour rappel, on calcule le centre de gravité entre deux classes de la manière suivante :
\begin{align*}
G=\frac{1}{n_1+n_2}(n_1 G_1+n_2G_2).
\end{align*}

**Question 4.** Créez une fonction `indice` qui prend en arguments
- deux arrays `G1`, `G2` qui sont les centres de gravité des deux classes,
- deux nombres `n1` et `n2` qui indiquent le nombre d'individus dans chaque classe,
- un nombre `n` qui indique le nombre total d'individus,

et renvoie l'indice d'agrégation entre ces deux classes qui est donné par la quantité

\begin{align*}
\frac{n_1n_2}{n(n_1+n_2)}\mathrm{dist}(G_1,G_2)^2.
\end{align*}

Testez votre fonction avec les données jouet et les indices d'agrégation fournis page 11 du poly. 

In [ ]:
# définition de la fonction indice

In [ ]:
# test avec les données jouet
X=array([[12,16,14],[14,13,15],[8,11,8],[10,9,6],[16,6,17]]) # rappel des données jouet

**Question 5.** Créez une fonction `agreg` qui prend en arguments
- un array `donnees` (chaque ligne $i$ de la matrice contient les coordonnées du centre de gravité de la classe $i$)
- un array `nbindiv` qui indique le nombre d'individus par classe (le terme $i$ de ce vecteur indique le nombre d'individu dans la classe $i$)

et renvoie une matrice contenant les indices d'agrégation entre toutes les paires de classes. La ligne $i$ et la colonne $j$ de la matrice renvoyée devra correspondre à l'indice d'agrégation entre les classes $i$ et $j$. 

Testez votre fonction avec les données jouet.

In [ ]:
# définition de la fonction agreg

In [ ]:
# test avec les données jouet

**Question 6.** On a défini ci-dessous une fonction `minagreg` qui prend les mêmes entrées que `agreg` (et utilise cette fonction) et renvoie :
- l'indice d'agrégation le plus faible entre toutes les paires de classes,
- les numéros $i$ et $j$ des deux classes correspondantes avec $i\leq j$.

Exécutez les lignes de commande ci-dessous et testez la fonction `minagreg` avec les données jouet.

(Indications qui aident à comprendre les lignes de commande : 

1) il faut faire en sorte de ne pas prendre en compte l'indice d'agrégation entre une classe et elle-même qui est toujours nul (cf les 0 sur la diagonale)! Pour cela, on a ajouté à la matrice d'agrégation `Mat` une matrice égale à `2*m*identity(p)` où $m$ est la plus grande valeur de `Mat` et `identity(p)` est la matrice identité de format $p\times p$.

2) étant donnée une matrice M, on peut déterminer la valeur maximale de cette matrice via `M.max()` et la valeur minimale via `M.min()`. Pour savoir où se trouve l'élément réalisant ce minimum, on utilise la commande `argmin`. Cette commande donne le numéro de l'élément en parcourant la matrice ligne par ligne. Si la matrice est de taille $(p\times p)$, on détermine le numéro de ligne en effectuant la division euclidienne par le nombre de colonnes (commande `//p`) et le numéro de colonne est alors donné par le reste de cette division euclidienne (commande `%p`).)

In [ ]:
def minagreg(donnees, nbindiv):
    c=donnees.shape[0] # nombre de classes
    Mat=agreg(donnees, nbindiv)
    m = Mat.max() # maximum de la matrice d'agrégation
    Mat = Mat+2*m*identity(c) # on remplace les zeros de la diagonale par la valeur 2*max
    mini = Mat.min() # minimum de la matrice d'agregation en dehors de la digonale
    arg=Mat.argmin() # la position de ce minimum, en parcourant la matrice ligne par ligne
    i = arg//c # ligne où se trouve le min
    j = arg%c # colonne où se trouve le min
    ordo = sort([i,j])  # on ordonne i et j 
    return mini, ordo[0], ordo[1]

In [ ]:
# test avec les données jouet

**Question 7.** Tester à présent cette fonction sur les données normalisées des dépenses de l'état.

**Question 8.** On donne ci-dessous la fonction `ward` qui prend en entrées
- un array de données `donnees`;
- une liste `noms` qui contient les noms des individus (chaque nom se trouvant dans un ensemble, c'est-à-dire entre accolades);

et renvoie

- un array `classe` qui contient la classe finale (contenant donc tous les individus), décrite par l'ensemble des noms des individus s'y trouvant, et le nombre total d'individus qui s'y trouvent;
- un array `indices` qui contient l' indice d'agrégation des deux classes fusionnées pour chaque étape;
- un array `clustering` qui contient les classes existantes à chaque étape (où les classes sont décrites par l'ensemble des noms des individus s'y trouvant).

Testez la fonction `ward` sur les données jouet et comparez à la classification décrite dans le poly. 

In [ ]:
def ward(donnees,label):
    n,p=donnees.shape
    classe=[]
    clustering=[label]
    for i in range(n):
        classe.append([label[i],1])
    indices=[]
    for k in range(n-1):
        nbindiv=array(classe)[:,1]
        mini,i,j=minagreg(donnees,nbindiv)
        indices.append(mini)
        nvelle_ligne=bary(donnees[i,:],donnees[j,:],nbindiv[i],nbindiv[j])
        donnees=delete(append(donnees,[nvelle_ligne],axis=0),[i,j],axis=0)
        classe.append([classe[i][0].union(classe[j][0]),classe[i][1]+classe[j][1]])
        del classe[j]
        del classe[i]
        nvelle_classe=[]
        for l in range(n-1-k):
            nvelle_classe.append(classe[l][0])
        clustering.append(nvelle_classe)
    return classe,indices,clustering

In [ ]:
# test avec les données jouet
ward(Z,[{'A'},{'B'},{'C'},{'D'},{'E'}])

**Question 9.** Appliquez la fonction `ward` aux données des dépenses de l'état. Affichez les 4 clusters apparaissant à l'étape $n-4$. 

**Question 9.** Tracez les indices d'agrégation obtenus à chaque étape de l'algorithme de Ward. A quel moment observez vous le premier saut de l'indice d'agrégation ?

**Bonus 1.** A partir de `clustering` renvoyé par la fonction `ward`, on peut obtenir les quatre clusters d'années calculés avec l'algorithme de Ward. On peut également représenter à nouveau les points-individus dans le plan principal, en attribuant à chaque individu une couleur différente en fonction de sa classe.

In [ ]:
# affichage des quatre clusters
quatre_classes=clustering[n-4]
for i in range(4):
    print(quatre_classes[i])
# représentation des points-individus dans le plan principal avec des couleurs différentes suivant les classes
figure(figsize=(10,10))
for i in range(n):
    if (annees[i] in quatre_classes[0]):
        scatter(C1[i], C2[i], color='red')
        annotate(annees[i],(C1[i], C2[i]))
    elif (annees[i] in quatre_classes[1]):
        scatter(C[i,0], C[i,1], color='blue')
        annotate(annees[i],(C[i,0], C[i,1]))
    elif (annees[i] in quatre_classes[2]):
        scatter(C[i,0], C[i,1], color='orange')
        annotate(annees[i],(C[i,0], C[i,1]))
    else:
        scatter(C[i,0], C[i,1], color='green')
        annotate(annees[i],(C[i,0], C[i,1]))
axhline(y=0, lw=1, color='k') # axe y=0
axvline(x=0, lw=1, color='k') # axe x=0
show()

**Bonus 2.** Affichage des étapes de l'algorithme de Ward sous forme de dendrogramme à l'aide de la bibliothèque scipy.


In [ ]:
import scipy.cluster.hierarchy as hi 
classif=hi.ward(data_norm)
figure(figsize=(8,6))
hi.dendrogram(classif, labels=annees)
show()

**Bonus 3.** Classification des données des dépenses de l'état grâce à l'algorithme des K-means en utilisant la bibliothèque Scikit-learn. La méthode des K-means est implémentée dans sklearn sous le nom de `cluster.KMeans`(documentation disponible ici https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). 

Ici, on a choisi `random` comme inititialisation et choisi une classification en 4 classes. En recommençant l'exécution, on peut constater que l'algo ne fournit pas toujours la même réponse. 

In [ ]:
from sklearn import * # chargement de la librairie Scikit-learn
classif2=cluster.KMeans(n_clusters=4,init='random').fit(data_norm)
print(classif2.labels_)
print(classif2.inertia_)
for i in range(4):
    print('Classe',i,'=',annees[classif2.labels_==i])